In [13]:
import z3
import SATfunctions2 as SATf
from scipy.optimize import minimize
from math import log,ceil

def read_input_file(file_path):
    with open(file_path, 'r') as file:
        m = int(file.readline().strip())
        n = int(file.readline().strip())
        #s = [0 for _ in range(m)]
        s = list(map(int, file.readline().strip().split()))
        w = list(map(int, file.readline().strip().split()))

        # Inizializza la matrice delle distanze D
        D = [[0 for _ in range(n+1)] for _ in range(n+1)]

        # Leggi i valori della matrice delle distanze dalla restante parte del file
        for i in range(n+1):
            row_values = list(map(int, file.readline().strip().split()))
            D[i] = row_values

    #U vettore di n variabili (1 e n) con valore da 1 a n

    return m, n, s, w, D

In [14]:
def SAT_mcp(m, n, s, w, D):
    mb = SATf.int2boolList(m)
    nb = SATf.int2boolList(n)
    sb = [SATf.int2boolList(s[i]) for i in range(m)]
    wb = [SATf.int2boolList(w[i]) for i in range(n)]
    Db = [[SATf.int2boolList(D[i][j]) for j in range(n+1)] for i in range(n+1)]
    # Inizializza il solver Z3
    solver = z3.Solver()

    x = [[z3.Bool(f'x_{i}_{j}') for j in range(len(mb))] for i in range(n)]
    tour = [[z3.Bool(f'tour_{i}_{j}') for j in range(len(nb))] for i in range(n)]
    #Domains
    for i in range(n):
        solver.add(SATf.at_least_one_T(x[i]))
        solver.add(SATf.at_least_one_T(tour[i]))
        solver.add(SATf.lte(x[i],mb))
        solver.add(SATf.lte(tour[i],SATf.sum_b_list([SATf.eq(x[i],x[k]) for k in range(n)])))

    #C1     weight constraint
    for i in range(m):
        solver.add(SATf.gte(sb[i],SATf.sum_b_list([SATf.enable(wb[j],SATf.eq(SATf.int2boolList(i+1),x[j])) for j in range(n)])))
        solver.add(SATf.gte(sb[0],SATf.sum_b_list([SATf.enable(wb[0],SATf.eq(SATf.int2boolList(i+1),x[0]))])))
    
    #C2   a courier cannot deliver two items at the same time  
    for i in range(n):
        for j in range(i+1,n):
            solver.add(z3.Or(SATf.ne(x[i],x[j]),SATf.ne(tour[i],tour[j])))

    #loss
    lb = min(D[n])+min([D[i][n] for i in range(n+1)])
    ub = sum([max(D[i]) for i in range(n+1)])
    dist = [[] for _ in range(m)]    #list of distances of each courier
    loss = [z3.Bool(f'loss_{j}') for j in range(ceil(log(ub,2)))]
    for i in range(m):
        dist[i]=SATf.sum_b_list([SATf.enable(Db[j][k],z3.And(SATf.eq(x[j],SATf.int2boolList(i+1)),SATf.eq(x[k],SATf.int2boolList(i+1)),SATf.eq(SATf.sum_b(x[j],[True]),x[k]))) for j in range(n) for k in range(n)])
        dist[i]=SATf.sum_b(dist[i],SATf.sum_b_list([SATf.enable(Db[n][j],z3.And(SATf.eq(SATf.int2boolList(i+1),x[j]),SATf.eq([True],tour[j]))) for j in range(n)]))
        dist[i]=SATf.sum_b(dist[i],SATf.sum_b_list([SATf.enable(Db[n][j],z3.And(SATf.eq(SATf.int2boolList(i+1),x[j]),SATf.eq(SATf.sum_b_list([SATf.eq(x[i],x[k]) for k in range(n)]),tour[j]))) for j in range(n)]))
        solver.add(SATf.gte(loss,dist[i]))
    
    solver.add(SATf.at_least_one_T([SATf.eq(loss,dist[i]) for i in range(m)]))
    solver.add(SATf.gte(loss,SATf.int2boolList(lb)))
    solver.add(SATf.gte(SATf.int2boolList(ub),loss))

    if solver.check() == z3.sat:
        model = solver.model()
        print("Sat:")
        print(model)
    else:
        print("Unsat")

In [15]:
m,n,s,w,D = read_input_file('instance.txt')
print(m,n,s,w,D)
SAT_mcp(m, n, s, w, D)

3 7 [15, 10, 7] [3, 2, 6, 8, 5, 4, 4] [[0, 3, 3, 6, 5, 6, 6, 2], [3, 0, 4, 3, 4, 7, 7, 3], [3, 4, 0, 7, 6, 3, 5, 3], [6, 3, 7, 0, 3, 6, 6, 4], [5, 4, 6, 3, 0, 3, 3, 3], [6, 7, 3, 6, 3, 0, 2, 4], [6, 7, 5, 6, 3, 2, 0, 4], [2, 3, 3, 4, 3, 4, 4, 0]]
Sat:
[x_3_0 = True,
 xge_688f9be3-f558-4adb-b03d-bc1e73a42c84 = False,
 tour_5_2 = True,
 xge_628cfd0d-eb84-4c7c-9e2b-7cdfd10bcb01 = True,
 xge_e7815ac5-d64e-4940-956f-615f760fc9ee = True,
 xge_ed2df3ea-ff79-488f-9468-7e8be787abd2 = True,
 xge_6f92b783-9e5a-4d8d-a455-4f27547fd41c = True,
 x_5_0 = False,
 tour_6_0 = False,
 tour_0_0 = False,
 xge_3ac70cc4-664e-4fc7-a2d7-f18c2d2f8056 = True,
 xge_15733750-a50a-4d03-8285-35ddb41fbc96 = True,
 x_0_1 = True,
 xge_b82315d6-f9d7-4d4d-8b81-36bcf71228e0 = False,
 tour_3_0 = False,
 xge_43753a1e-cd36-43c0-b1a2-a0ab26e785e5 = True,
 xge_f09d44a3-bb26-4ae2-bf19-2300a0e0ff67 = True,
 xge_51af88ae-9e40-40fc-b1ed-bfc956dcdb42 = False,
 tour_3_1 = False,
 loss_4 = True,
 xge_22b9d7e0-f199-476e-ad3a-3e64e84264